In [1]:
import collections.abc
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time

import pandas as pd
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import sys

# soup error 방지
import collections
if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

C:\Users\LG\AppData\Local\Temp\ipykernel_12248\1179179661.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if not hasattr(collections, 'Callable'):


In [ ]:
# 데이터 1만개 크롤링
for i in range(1,1000):  
    url =   f'https://kin.naver.com/search/list.naver?query=%EC%9D%B4%EB%B9%84%EC%9D%B8%ED%9B%84%EA%B3%BC&page={i}'
    html   =   urlopen(url)
    soup   =   BeautifulSoup(html.read(),   'html.parser')
    links = soup.find('ul', class_='basic1').find_all('a')


    links_list = []
    for link in links:
        if ('https://kin.naver.com/qna/detail.naver?d1id' in link['href']) and (link['href'] not in links_list):
            links_list.append(link['href'])

    # links_list = [link['href'] for link in links if 'https://kin.naver.com/qna/detail.naver?d1id' in link['href']]

    # 링크 출력 (최대 10개)
    # for href in links_list:
    #     print(f"{href}")

    for idx, url in enumerate(links_list):
        response = requests.get(url)

        # HTML을 BeautifulSoup으로 파싱
        soup = BeautifulSoup(response.text, 'html.parser')
        soup = soup.find('div', {'class':'questionDetail'})
        soup = soup.get_text(separator=' ').strip()
        FILE_PATH = '../../../LocalData/nlp/pj_data/'
        f = open(FILE_PATH+f"{i}, {idx}.txt", 'w')
        # print(i, idx, soup)
        f.write(soup)

In [ ]:
textDF = pd.DataFrame(columns=['txt','label'])

# 본인 데이터 파일 지정
TRAIN_PATH = '../../../LocalData/nlp/otolaryngology_data/'
files = os.listdir(TRAIN_PATH)
idx = 0
for file in files:
    with open(TRAIN_PATH+file, mode='r', encoding='utf-8') as f:
        text=f.read()
        text = re.sub('[^가-힣]+',' ',text)
        textDF.loc[idx] = [text,1]
        idx+=1
        print(idx)
        # 전체 인덱스 지정용 idx

# 내과
i_idx=1
TRAIN_PATH = '../../../LocalData/nlp/medicine_data/'
files = os.listdir(TRAIN_PATH)
for file in files:
    with open(TRAIN_PATH+file, mode='r', encoding='utf-8') as f:
        text=f.read()
        text = re.sub('[^가-힣]+',' ',text)
        textDF.loc[idx] = [text,0]
        i_idx+=1
        idx+=1
        print(f"idx : {idx}, n_idx : {i_idx}")
        if i_idx == 3300: break
        # i_idx 는 inner_data용 idx 3300개만 뽑기 위한 용도, 아래 n_idx, m_idx 또한 같은 용도. 참고

# 정신과용
m_idx=1
TRAIN_PATH = '../../../LocalData/nlp/psychiatry_data/'
files = os.listdir(TRAIN_PATH)
for file in files:
    with open(TRAIN_PATH+file, mode='r', encoding='utf-8') as f:
        text=f.read()
        text = re.sub('[^가-힣]+',' ',text)
        textDF.loc[idx] = [text,0]
        m_idx+=1
        idx+=1
        print(f"idx : {idx}, n_idx : {m_idx}")
        if m_idx == 3300: break


# 안과용
n_idx=1
TRAIN_PATH = '../../../LocalData/nlp/eyes_data/'
files = os.listdir(TRAIN_PATH)
for file in files:
    with open(TRAIN_PATH+file, mode='r', encoding='utf-8') as f:
        text=f.read()
        text = re.sub('[^가-힣]+',' ',text)
        textDF.loc[idx] = [text,0]
        n_idx+=1
        idx+=1
        print(f"idx : {idx}, n_idx : {n_idx}")
        if n_idx == 3300: break


# 종료시 알림
import winsound as sd
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 300     # 1000 ms ==1second
    sd.Beep(fr, du)
beepsound()

In [3]:
# 문장 분류 모델
from torch import nn

class SentenceClassifier(nn.Module):
    def __init__(
            self,
            n_vocab,
            hidden_dim,
            embedding_dim,
            n_layers,
            dropout=0.5,
            bidirectional=True,
            model_type="lstm"
    ):
        super().__init__()

        self.embedding=nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )
        if model_type =="rnn":
            self.model=nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type =="lstm":
            self.model=nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        if bidirectional:
            self.classifier=nn.Linear(hidden_dim * 2,1)
        else:
            self.classifier=nn.Linear(hidden_dim, 1)
        self.dropout=nn.Dropout(dropout)
    
    def forward(self, inputs):
        embeddings=self.embedding(inputs)
        output, _=self.model(embeddings)
        last_output=output[:, -1, :]
        last_output=self.dropout(last_output)
        logits=self.classifier(last_output)
        return logits

In [4]:
# 데이터세트 불러오기
train_data=textDF.sample(frac=0.9, random_state=42)
test_data=textDF.drop(train_data.index)

print(train_data.head(5).to_markdown())
print("Training Data Size :", len(train_data))
print("Testing Data Size :", len(test_data))

|       | txt                                                                                                                                                                                                  |   label |
|------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------:|
|  1430 | 이비인후과 집 가까이 있는곳 가려는데 팔달시장에 수이비인후과 소아청소년만 보나요                                                                                                                     |       1 |
| 16400 | 고 학생인데 중학생때부터 집중도 통 안되고 망상에 빠지는 경우가 너무 많아서 검사를 한 번 해보려고 하는데요 검색해보니 판정받으면 정신과 기록이 남는다 하던데 혹시 이게 나중에 악영향을 끼칠 수 있나요 |       0 |
|  8144 | 후비루 치료 잘하는 이비인후과                                                                                                                                                                        |       1 |
|  4610 | 이비인후과 집 가까이 있는곳 가려는데 팔달시장에 수

In [5]:
# 데이터 토큰화 및 단어 사전 구축
from konlpy.tag import Okt
from collections import Counter

def build_vocab(corpus, n_vocab, special_tokens):
    counter=Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab=special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

tokenizer=Okt()
train_tokens=[tokenizer.morphs(review) for review in train_data.txt]
test_tokens=[tokenizer.morphs(review) for review in test_data.txt]

vocab=build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id={token: idx for idx, token in enumerate(vocab)}
id_to_token={idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '이', '에', '가', '이비인후과', '를', '을', '는', '도']
5002


In [6]:
# 단어 사전 csv 저장
vocab_df = pd.DataFrame(columns=['key', 'value'])
vocab_df['key'] = token_to_id.keys()
vocab_df['value'] = token_to_id.values()
vocab_df.to_csv('../../../LocalData/nlp/my_vocab/nose_vocab.csv', encoding='utf-8')

In [7]:
# 정수 인코딩 및 패딩
import numpy as np

def pad_sequences(sequences, max_length, pad_value):
    result=list()
    for sequence in sequences:
        sequence=sequence[:max_length]
        pad_length=max_length-len(sequence)
        padded_sequence=sequence+[pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

unk_id=token_to_id['<unk>']
train_ids=[
    [token_to_id.get(token, unk_id) for token in review] for review in train_tokens
]
test_ids=[
    [token_to_id.get(token, unk_id) for token in review] for review in test_tokens
]

max_length=32
pad_id=token_to_id["<pad>"]
train_ids=pad_sequences(train_ids, max_length, pad_id)
test_ids=pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[  5 209 391 146 143 101 344 137 407   3  30   5 180 264  51 392  13   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ 850 2695    3 1097    2    9  391    1  195 1021 1791    1  793    5
   44 1053    9 1280 2745  147    1    1   68   32  508    1 2116 2052
 2869    1  196 1403]


In [8]:
# 데이터로더 적용
import torch
from torch.utils.data import TensorDataset, DataLoader

train_ids=torch.tensor(train_ids)
test_ids=torch.tensor(test_ids)

train_labels=torch.tensor(train_data.label.values, dtype=torch.float32)
test_labels=torch.tensor(test_data.label.values, dtype=torch.float32)

train_dataset=TensorDataset(train_ids, train_labels)
test_dataset=TensorDataset(test_ids, test_labels)

train_loader=DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=16, shuffle=False)

In [9]:
# 손실 함수와 최적화 함수 정의
from torch import optim

n_vocab=len(token_to_id)
hidden_dim=64
embedding_dim=128
n_layers=2

device="cuda" if torch.cuda.is_available() else "cpu"
classifier=SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
).to(device)
criterion=nn.BCEWithLogitsLoss().to(device)
optimizer=optim.RMSprop(classifier.parameters(), lr=0.001)

In [10]:
# 모델 학습 함수
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses=list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids=input_ids.to(device)
        labels=labels.to(device).unsqueeze(1)

        logits=model(input_ids)
        loss=criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval==0:
            print(f"Train Loss {step} : {np.mean(losses)}")

In [11]:
# 모델 테스트 함수
def test(model, datasets, criterion, device, min_loss, max_score, trigger, epoch):
    model.eval()
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits)>.5
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )
    t_loss = np.mean(losses)
    t_score = np.mean(corrects)

    SAVE_FILE = f'../../../LocalData/nlp/models/model_train_wbs_{epoch}_{t_score}.pth' 
    SAVE_MODEL = f'../../../LocalData/nlp/models/model_train_all_{epoch}_{t_score}.pth' 

    if min_loss < t_loss:
        trigger+=1
        print('loss 상승')
    else : 
        trigger=0
        min_loss = t_loss
    
    if max_score < t_score:
        torch.save(model.state_dict() ,SAVE_FILE)
        torch.save(model, SAVE_MODEL)
    print(f"Val Loss : {t_loss}, Val Accuracy : {t_score}")
    

In [13]:
epochs=100
interval=500

min_loss, max_score, trigger=100, -1, 0

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device, min_loss, max_score, trigger, epoch)

Train Loss 0 : 1.5970624822614354e-10


KeyboardInterrupt: 

In [14]:
# 학습된 모델로부터 임베딩 추출
token_to_embedding=dict()
embedding_matrix=classifier.embedding.weight.detach().cpu().numpy()

for word, emb in zip(vocab, embedding_matrix):
    token_to_embedding[word]=emb

token=vocab[1000]
print(token, token_to_embedding[token])

스펜서 [-1.17756     1.3826123  -0.9393518  -0.5108446   0.20590624  0.4730828
 -0.15738548  0.91913676 -1.5182947   1.3937839   1.1660508  -1.7804289
  1.0866077   0.35119864 -0.62764955  0.8202577  -0.8918364   0.2951318
  0.7314091  -0.09585405 -0.5506177  -0.02704389 -0.89259285  0.42719123
  0.21297316  1.4330459   0.35770783  0.56020284  1.757232    1.271189
 -0.40390894 -0.58911556  0.97709715 -1.5738097  -0.14241177  0.19763255
 -2.2133343   0.5757457   1.2171783   0.35146493 -0.36045954  0.710397
  0.3819382  -0.31749526 -1.0158403  -0.8202974  -0.2042847  -0.5847889
 -0.3832918   0.17157674  0.38590717 -1.8761234  -0.07825271  1.8193064
 -0.5564737  -0.6393762  -0.23808424 -0.8561183   0.25855374 -1.1698513
  0.3700656   0.21001329  0.6314755  -1.3975614   1.2847075   0.96625614
 -0.59797823  1.1720486   0.1980955  -1.2359601  -1.2676605   0.7555174
 -0.5192259   0.6573489   0.88628775  0.9962074  -0.29819015 -0.19083454
 -0.34732282 -0.6416058  -0.02120762  1.218884   -2.206403

In [20]:
LABEL_TRANSLATE = {0:'이비인후과는 아닌듯...', 1:'이비인후과'}

In [54]:
def predict(model, x_data, y_data):
    # 텐서 하나
    # y_data의 차원을 확인하고 unsqueeze가 필요한지 확인
    if len(y_data.shape) == 1:  # 1D 텐서인 경우
        y_data = y_data.unsqueeze(1)  # (N,) -> (N, 1)
    
    y_data = y_data.float()  # float형으로 변환
    x_data = x_data.unsqueeze(1)
    with torch.no_grad():
        # 예측 수행
        pred = model(x_data)
        pred=torch.sigmoid(pred)
        # 0.5 이상이면 1, 미만이면 0으로 변환
        pred_labels = torch.argmax((pred >= 0.5).int())
        pred_labels = [LABEL_TRANSLATE[int(label)] for label in pred_labels.flatten()]
        real_labels = [LABEL_TRANSLATE[int(label)] for label in y_data.flatten()]

    return pred_labels, real_labels

In [55]:
lstm_model=torch.load('../../../LocalData/nlp/models/model_train_all_9_0.9904474610356964.pth', weights_only=False)

In [61]:
print(test_dataset.tensors[0][5])

tensor([ 65,   8, 111,   8,  23,  94, 123,   7,  97,  25, 325,  99,  50,   5,
         11,  94, 123,   7,  97, 110, 182,  11,   4, 124, 303,  57,  16,  36,
        329,   7, 355, 334], dtype=torch.int32)


In [56]:
num_list=[5,8,13,17,60,77,404,800]

for num in num_list:
    pred_label, real_label = predict(lstm_model, test_dataset.tensors[0][num], test_dataset.tensors[1][num])

    print(f"예측: {pred_label[0]}")
    print(f"결과: {real_label[0]}\n")

예측: 이비인후과
결과: 이비인후과

예측: 이비인후과
결과: 이비인후과

예측: 이비인후과
결과: 이비인후과

예측: 이비인후과
결과: 이비인후과

예측: 이비인후과
결과: 이비인후과

예측: 이비인후과
결과: 이비인후과

예측: 이비인후과
결과: 이비인후과

예측: 이비인후과
결과: 이비인후과

